In [102]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format
import math
from sklearn import neighbors
import statsmodels.formula.api as smf
from sklearn.model_selection import cross_val_score
from scipy import stats

# 3.3.4 Logistic Regression
Task
Now that you have two new regression methods at your fingertips, it's time to give them a spin. In fact, for this challenge, let's put them together! Pick a dataset of your choice with a binary outcome and the potential for at least 15 features. If you're drawing a blank, the crime rates in 2013 dataset has a lot of variables that could be made into a modelable binary outcome.

Engineer your features, then create three models. Each model will be run on a training set and a test-set (or multiple test-sets, if you take a folds approach). The models should be:

Vanilla logistic regression, Ridge logistic regression, Lasso logistic regression

In your report, evaluate all three models and decide on your best. Be clear about the decisions you made that led to these models (feature selection, regularization parameter selection, model evaluation criteria) and why you think that particular model is the best of the three. Also reflect on the strengths and limitations of regression as a modeling approach. Were there things you couldn't do but you wish you could have done?

In [103]:
df = pd.read_csv('real_property_residential_characteristics.csv')
df.head(20)

,PIN,SCHEDNUM,CD,OFCARD,OWNER,CO_OWNER,OWNER_NUM,OWNER_DIR,OWNER_ST,OWNER_TYPE,...,UNITS,ASMT_APPR_LAND,TOTAL_VALUE,ASDLAND,ASSESS_VALUE,ASMT_TAXABLE,ASMT_EXEMPT_AMT,NBHD_1,NBHD_1_CN,LEGL_DESCRIPTION
0,160431419,4100122000,1,1,"REITHER,JOHN FAMILY TRUST",NaN,17675,E,64TH,AVE,...,1,0.000,264000.000,0.000,19008.000,19010.000,0.000,4,GATEWAY N,T3 R66 S4 SW/4 E 660FT EXC S 30FT OF SD SEC ...
1,160431427,4100123000,1,1,"GARRISON,DONALD H",NaN,17675,E,64TH,AVE,...,1,0.000,208200.000,0.000,14990.000,14990.000,0.000,4,GATEWAY N,T3 R66 S4 SW/4 E 660FT EXC S 30FT OF SD SEC ...
2,163087844,10201002000,1,1,VACHOVSKY FAMILY TRUST,NaN,6151,NaN,AVENIDA PICACHO,NaN,...,1,40100.000,216200.000,2887.000,15566.000,15570.000,0.000,4,GATEWAY N,FIRST CREEK FLG NO 1 B1 L2
3,163087852,10201003000,1,1,BISCAY REALTY LLC,NaN,10798,NaN,LISMORE,WAY,...,1,25300.000,211100.000,1822.000,15200.000,15200.000,0.000,4,GATEWAY N,FIRST CREEK FLG NO 1 B1 L3
4,163087861,10201004000,1,1,"SIERRA,JUAN ROBERTO",NaN,5837,NaN,BISCAY,ST,...,1,25300.000,188900.000,1822.000,13601.000,13600.000,0.000,4,GATEWAY N,FIRST CREEK FLG NO 1 B1 L4
5,163087879,10201005000,1,1,O AND P ENTERPRISES LLC,NaN,3670,NaN,VESTAL,LOOP,...,1,25300.000,210300.000,1822.000,15142.000,15140.000,0.000,4,GATEWAY N,FIRST CREEK FLG NO 1 B1 L5
6,163087887,10201006000,1,1,"MCCLEAF,BARBARA",NaN,5833,NaN,BISCAY,ST,...,1,25300.000,210300.000,1822.000,15142.000,15140.000,0.000,4,GATEWAY N,FIRST CREEK FLG NO 1 B1 L6
7,163087895,10201007000,1,1,"MURPHY,JOSEPH",NaN,5827,NaN,BISCAY,ST,...,1,25300.000,202200.000,1822.000,14559.000,14560.000,0.000,4,GATEWAY N,FIRST CREEK FLG NO 1 B1 L7
8,163087909,10201008000,1,1,"CRAVEN,BRANDEN",NaN,5825,NaN,BISCAY,ST,...,1,25300.000,189000.000,1822.000,13608.000,13610.000,0.000,4,GATEWAY N,FIRST CREEK FLG NO 1 B1 L8
9,163087917,10201009000,1,1,"SOSA-IBANEZ,PERLA",NaN,5823,NaN,BISCAY,ST,...,1,32700.000,222000.000,2354.000,15984.000,15980.000,0.000,4,GATEWAY N,FIRST CREEK FLG NO 1 B1 L9


In [104]:
df.count()

PIN                 206053
SCHEDNUM            206053
CD                  206053
OFCARD              206053
OWNER               206053
CO_OWNER              6913
OWNER_NUM           206053
OWNER_DIR           119823
OWNER_ST            206053
OWNER_TYPE          199948
OWNER_APT            41328
OWNER_CITY          206053
OWNER_STATE         206053
OWNER_ZIP           206044
SITE_NBR            206053
SITE_DIR            198026
SITE_NAME           206053
SITE_MODE           205896
SITE_MORE            49942
TAX_DIST            206053
PROP_CLASS          206052
PROPERTY_CLASS      205845
ZONE10              206053
D_CLASS_CN          206053
LAND_SQFT           205390
AREA_ABG            206053
BSMT_AREA           206053
FBSMT_SQFT          206053
GRD_AREA            206053
STORY               206053
STYLE_CN            194265
BED_RMS             206053
FULL_B              194265
HLF_B               194265
CCYRBLT             194265
CCAGE_RM            194265
UNITS               206053
A

In [105]:
#combine bathroom types into one 'total bathrooms' column
df['Bathrooms'] = df['FULL_B']+df['HLF_B']
df = df.drop(['FULL_B', 'HLF_B'], axis=1)

In [106]:
#remove outlier
tvmax = df.loc[df['TOTAL_VALUE'].idxmax()]
df = df[df.TOTAL_VALUE !=  6.216140e+07]
df['TOTAL_VALUE'].describe()

count     206051.000
mean      395699.025
std       324975.579
min          100.000
25%       232400.000
50%       327900.000
75%       474300.000
max     12912800.000
Name: TOTAL_VALUE, dtype: float64

In [107]:
#Remove last four digits and the dash of the zip code
df['OWNER_ZIP'] = df['OWNER_ZIP'].str[:5]
df['OWNER_ZIP'].head(20)

0     80249
1     80249
2     92067
3     80126
4     80249
5     80023
6     80249
7     80249
8     80249
9     80249
10    80249
11    80249
12    80249
13    80249
14    80249
15    21113
16    80249
17    80249
18    80205
19    80027
Name: OWNER_ZIP, dtype: object

In [108]:
df.dtypes

PIN                   int64
SCHEDNUM              int64
CD                    int64
OFCARD                int64
OWNER                object
CO_OWNER             object
OWNER_NUM            object
OWNER_DIR            object
OWNER_ST             object
OWNER_TYPE           object
OWNER_APT            object
OWNER_CITY           object
OWNER_STATE          object
OWNER_ZIP            object
SITE_NBR              int64
SITE_DIR             object
SITE_NAME            object
SITE_MODE            object
SITE_MORE            object
TAX_DIST             object
PROP_CLASS          float64
PROPERTY_CLASS       object
ZONE10               object
D_CLASS_CN           object
LAND_SQFT           float64
AREA_ABG              int64
BSMT_AREA             int64
FBSMT_SQFT            int64
GRD_AREA              int64
STORY                 int64
STYLE_CN             object
BED_RMS               int64
CCYRBLT             float64
CCAGE_RM            float64
UNITS                 int64
ASMT_APPR_LAND      

In [109]:
#Double Check there is no missing data
def num_missing(x):
  return sum(x.isnull())

#Applying per column:
print("Missing values per column:")
print(df.apply(num_missing, axis=0))

Missing values per column:
PIN                      0
SCHEDNUM                 0
CD                       0
OFCARD                   0
OWNER                    0
CO_OWNER            199139
OWNER_NUM                0
OWNER_DIR            86230
OWNER_ST                 0
OWNER_TYPE            6105
OWNER_APT           164725
OWNER_CITY               0
OWNER_STATE              0
OWNER_ZIP                9
SITE_NBR                 0
SITE_DIR              8027
SITE_NAME                0
SITE_MODE              157
SITE_MORE           156110
TAX_DIST                 0
PROP_CLASS               1
PROPERTY_CLASS         208
ZONE10                   0
D_CLASS_CN               0
LAND_SQFT              663
AREA_ABG                 0
BSMT_AREA                0
FBSMT_SQFT               0
GRD_AREA                 0
STORY                    0
STYLE_CN             11787
BED_RMS                  0
CCYRBLT              11787
CCAGE_RM             11787
UNITS                    0
ASMT_APPR_LAND           1
T

In [110]:
#remove missing values with the mean for bathrooms
df['Bathrooms'] = df['Bathrooms'].fillna(df['Bathrooms'].mean())
print("Missing values per column:")
print(df.apply(num_missing, axis=0))

Missing values per column:
PIN                      0
SCHEDNUM                 0
CD                       0
OFCARD                   0
OWNER                    0
CO_OWNER            199139
OWNER_NUM                0
OWNER_DIR            86230
OWNER_ST                 0
OWNER_TYPE            6105
OWNER_APT           164725
OWNER_CITY               0
OWNER_STATE              0
OWNER_ZIP                9
SITE_NBR                 0
SITE_DIR              8027
SITE_NAME                0
SITE_MODE              157
SITE_MORE           156110
TAX_DIST                 0
PROP_CLASS               1
PROPERTY_CLASS         208
ZONE10                   0
D_CLASS_CN               0
LAND_SQFT              663
AREA_ABG                 0
BSMT_AREA                0
FBSMT_SQFT               0
GRD_AREA                 0
STORY                    0
STYLE_CN             11787
BED_RMS                  0
CCYRBLT              11787
CCAGE_RM             11787
UNITS                    0
ASMT_APPR_LAND           1
T

In [111]:
df.dtypes

PIN                   int64
SCHEDNUM              int64
CD                    int64
OFCARD                int64
OWNER                object
CO_OWNER             object
OWNER_NUM            object
OWNER_DIR            object
OWNER_ST             object
OWNER_TYPE           object
OWNER_APT            object
OWNER_CITY           object
OWNER_STATE          object
OWNER_ZIP            object
SITE_NBR              int64
SITE_DIR             object
SITE_NAME            object
SITE_MODE            object
SITE_MORE            object
TAX_DIST             object
PROP_CLASS          float64
PROPERTY_CLASS       object
ZONE10               object
D_CLASS_CN           object
LAND_SQFT           float64
AREA_ABG              int64
BSMT_AREA             int64
FBSMT_SQFT            int64
GRD_AREA              int64
STORY                 int64
STYLE_CN             object
BED_RMS               int64
CCYRBLT             float64
CCAGE_RM            float64
UNITS                 int64
ASMT_APPR_LAND      

In [112]:
#drop columns that are objects and are unnecessary to predicting total value
df = df.drop(['LEGL_DESCRIPTION','NBHD_1_CN', 'STYLE_CN', 'SITE_DIR','SITE_NAME','SITE_MODE', 'SITE_MORE', 'PROPERTY_CLASS','ZONE10',
             'OWNER',
'CO_OWNER',
'OWNER_NUM', 
'OWNER_DIR',
'OWNER_ST', 
'OWNER_TYPE' ,
'OWNER_APT' ,
'OWNER_CITY' ,
'OWNER_STATE'],1 )



In [113]:
df = df.drop(['PIN', 'TAX_DIST'],1)

In [114]:
df = df.drop(['SCHEDNUM','CD','OFCARD'],1)

In [115]:
#convert D_CLASS_CN to numeric
df = pd.concat([df, pd.get_dummies(df['D_CLASS_CN'])], axis=1); df

,OWNER_ZIP,SITE_NBR,PROP_CLASS,D_CLASS_CN,LAND_SQFT,AREA_ABG,BSMT_AREA,FBSMT_SQFT,GRD_AREA,STORY,...,TAX MED IMPS@STAPLETON,TAX SMALL IMPS@STAPLETON,VCNT LAND,VCNT LAND - RES RATIO,VCNT LAND B-8 ZONE,VCNT LAND I-0 ZONE,"VCNT LAND R-2, RS-2 ZONE","VCNT LAND R-3, R-3X ZONE",VCNT LAND R-X ZONE,VCNT LAND W/MINOR STR
0,80249,17675,1112.000,SINGLE FAMILY,0.000,1269,1269,634,0,1,...,0,0,0,0,0,0,0,0,0,0
1,80249,17675,1112.000,SINGLE FAMILY,0.000,1682,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,92067,5845,1114.000,ROWHOUSE,2760.000,1716,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
3,80126,5843,1114.000,ROWHOUSE,1840.000,1523,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
4,80249,5837,1114.000,ROWHOUSE,1840.000,1401,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
5,80023,5835,1114.000,ROWHOUSE,1840.000,1540,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
6,80249,5833,1114.000,ROWHOUSE,1840.000,1540,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
7,80249,5827,1114.000,ROWHOUSE,1840.000,1401,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
8,80249,5825,1114.000,ROWHOUSE,1840.000,1401,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
9,80249,5823,1114.000,ROWHOUSE,2300.000,1779,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0


In [116]:
#remove all remaining missing values
df = df.dropna()

In [117]:
#make sure all missing values are gone
print(df.apply(num_missing, axis=0))

OWNER_ZIP                    0
SITE_NBR                     0
PROP_CLASS                   0
D_CLASS_CN                   0
LAND_SQFT                    0
AREA_ABG                     0
BSMT_AREA                    0
FBSMT_SQFT                   0
GRD_AREA                     0
STORY                        0
BED_RMS                      0
CCYRBLT                      0
CCAGE_RM                     0
UNITS                        0
ASMT_APPR_LAND               0
TOTAL_VALUE                  0
ASDLAND                      0
ASSESS_VALUE                 0
ASMT_TAXABLE                 0
ASMT_EXEMPT_AMT              0
NBHD_1                       0
Bathrooms                    0
101 CONDO PKG                0
10S CONDO STORAGE            0
APT MISC, PKG, CLUBHOUSES    0
CONDOMINIUM                  0
DCL MISC RES IMPS            0
DENVER PARK                  0
DRY FARM LAND                0
GENERAL COMMON ELEMENTS      0
GROUP/BOARDING HOME-1 KIT    0
MISC IMPS-TIE BACK           0
MISC ROW

In [118]:
df = df[~df.OWNER_ZIP.str.contains("DENVE")]
df = df[~df.OWNER_ZIP.str.contains("M")]
df = df[~df.OWNER_ZIP.str.contains("#")]
df = df[~df.OWNER_ZIP.str.contains("V")]
#convert zip code to integer
df['OWNER_ZIP'] = df['OWNER_ZIP'].astype(int)

In [119]:
df = df.drop(['D_CLASS_CN'],1)

In [120]:
df.dtypes

OWNER_ZIP                      int64
SITE_NBR                       int64
PROP_CLASS                   float64
LAND_SQFT                    float64
AREA_ABG                       int64
BSMT_AREA                      int64
FBSMT_SQFT                     int64
GRD_AREA                       int64
STORY                          int64
BED_RMS                        int64
CCYRBLT                      float64
CCAGE_RM                     float64
UNITS                          int64
ASMT_APPR_LAND               float64
TOTAL_VALUE                  float64
ASDLAND                      float64
ASSESS_VALUE                 float64
ASMT_TAXABLE                 float64
ASMT_EXEMPT_AMT              float64
NBHD_1                         int64
Bathrooms                    float64
101 CONDO PKG                  uint8
10S CONDO STORAGE              uint8
APT MISC, PKG, CLUBHOUSES      uint8
CONDOMINIUM                    uint8
DCL MISC RES IMPS              uint8
DENVER PARK                    uint8
D

In [121]:
#remove spaces from column headings
df.columns = df.columns.str.strip().str.replace(' ', '_').str.replace('@', '_').str.replace('/', '_')
print(df.columns)
print(df.TAX_MED_IMPS_STAPLETON)

Index(['OWNER_ZIP', 'SITE_NBR', 'PROP_CLASS', 'LAND_SQFT', 'AREA_ABG',
       'BSMT_AREA', 'FBSMT_SQFT', 'GRD_AREA', 'STORY', 'BED_RMS', 'CCYRBLT',
       'CCAGE_RM', 'UNITS', 'ASMT_APPR_LAND', 'TOTAL_VALUE', 'ASDLAND',
       'ASSESS_VALUE', 'ASMT_TAXABLE', 'ASMT_EXEMPT_AMT', 'NBHD_1',
       'Bathrooms', '101_CONDO_PKG', '10S_CONDO_STORAGE',
       'APT_MISC,_PKG,_CLUBHOUSES', 'CONDOMINIUM', 'DCL_MISC_RES_IMPS',
       'DENVER_PARK', 'DRY_FARM_LAND', 'GENERAL_COMMON_ELEMENTS',
       'GROUP_BOARDING_HOME-1_KIT', 'MISC_IMPS-TIE_BACK', 'MISC_ROWHOUSE_IMPS',
       'MISC._SITE_IMPROVEMENTS', 'MOBILE_HOME_-_IMP_ONLY',
       'OTHER_REC_FACILITIES', 'PARKING_GARAGE_8_STORY', 'PRESCHOOLER_NURSERY',
       'ROWHOUSE', 'SINGLE_FAMILY', 'SINGLE_FAMILY_MISC_LIV',
       'SPECIAL_PURPOSE', 'TAX_MED_IMPS_STAPLETON', 'TAX_SMALL_IMPS_STAPLETON',
       'VCNT_LAND', 'VCNT_LAND_-_RES_RATIO', 'VCNT_LAND_B-8_ZONE',
       'VCNT_LAND_I-0_ZONE', 'VCNT_LAND_R-2,_RS-2_ZONE',
       'VCNT_LAND_R-3,_R-3X_ZO

In [124]:
#create binary outcome
df['VALUE>400K'] = df["TOTAL_VALUE"] >= 400000
print(df['VALUE>400K'].unique())
df['VALUE>400K'] = np.where(df['VALUE>400K']==True, 1, 0)
print(df['VALUE>400K'].value_counts())

[False  True]
0    120189
1     73874
Name: VALUE>400K, dtype: int64


In [125]:
df2 = df.drop(['TOTAL_VALUE'],1)

In [128]:
print(list(df2))

['OWNER_ZIP', 'SITE_NBR', 'PROP_CLASS', 'LAND_SQFT', 'AREA_ABG', 'BSMT_AREA', 'FBSMT_SQFT', 'GRD_AREA', 'STORY', 'BED_RMS', 'CCYRBLT', 'CCAGE_RM', 'UNITS', 'ASMT_APPR_LAND', 'ASDLAND', 'ASSESS_VALUE', 'ASMT_TAXABLE', 'ASMT_EXEMPT_AMT', 'NBHD_1', 'Bathrooms', '101_CONDO_PKG', '10S_CONDO_STORAGE', 'APT_MISC,_PKG,_CLUBHOUSES', 'CONDOMINIUM', 'DCL_MISC_RES_IMPS', 'DENVER_PARK', 'DRY_FARM_LAND', 'GENERAL_COMMON_ELEMENTS', 'GROUP_BOARDING_HOME-1_KIT', 'MISC_IMPS-TIE_BACK', 'MISC_ROWHOUSE_IMPS', 'MISC._SITE_IMPROVEMENTS', 'MOBILE_HOME_-_IMP_ONLY', 'OTHER_REC_FACILITIES', 'PARKING_GARAGE_8_STORY', 'PRESCHOOLER_NURSERY', 'ROWHOUSE', 'SINGLE_FAMILY', 'SINGLE_FAMILY_MISC_LIV', 'SPECIAL_PURPOSE', 'TAX_MED_IMPS_STAPLETON', 'TAX_SMALL_IMPS_STAPLETON', 'VCNT_LAND', 'VCNT_LAND_-_RES_RATIO', 'VCNT_LAND_B-8_ZONE', 'VCNT_LAND_I-0_ZONE', 'VCNT_LAND_R-2,_RS-2_ZONE', 'VCNT_LAND_R-3,_R-3X_ZONE', 'VCNT_LAND_R-X_ZONE', 'VCNT_LAND_W_MINOR_STR', 'VALUE>400K']


# Linear Regression

In [131]:
# Import model
from sklearn.linear_model import LogisticRegression


# Instantiate and set regularization coefficient to large value
lr = LogisticRegression(C=1e9)

# Define variables
x = df2.drop(['VALUE>400K'],1)
y = df2['VALUE>400K']

# Fit model
fit = lr.fit(x, y)

# Get results
print('Coefficient')
coef = fit.coef_
print(coef)
print(fit.intercept_)

# Get predictions
pred_y = lr.predict(x)

print('\nAccuracy by fraud status')
print(pd.crosstab(pred_y, y))

print('R²:')
print(lr.score(x, y))

lr_scores = cross_val_score(lr, x, y, cv=5)
print('\nPercentage accuracy')
print(lr_scores)
print('Mean:', lr_scores.mean())

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Coefficient
[[-6.28131836e-05  1.78469074e-05 -2.40220671e-03  1.54738699e-06
   1.37857385e-04 -7.29769544e-05  2.49574817e-04  3.16703847e-04
  -5.75130036e-06 -2.25327750e-05 -2.32047905e-02 -1.99916458e-04
  -1.49259067e-05  1.57777032e-04 -2.25264332e-03  1.32210613e-03
   5.45735089e-04  7.97916260e-04 -4.74287414e-04  1.67857159e-05
  -2.85277598e-14 -1.13972654e-13  0.00000000e+00  6.06588607e-07
  -5.35296717e-12  4.42819038e-08  0.00000000e+00  0.00000000e+00
  -7.48141776e-10  0.00000000e+00  1.80688979e-10  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -5.95915072e-07 -1.34485713e-05  0.00000000e+00 -3.07718176e-14
   0.00000000e+00 -2.13662527e-14 -3.22772551e-13 -9.85858481e-12
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   5.59547076e-08  0.00000000e+00]]
[-1.33382443e-05]

Accuracy by fraud status
VALUE>400K       0      1
row_0                    
0           119413    771
1              776  73103
R²:
0.992028

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework


Percentage accuracy
[0.988947   0.97866694 0.99211604 0.9950532  0.98211847]
Mean: 0.9873803325532329


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


# Logistic Regression

In [137]:
# Define the training and test sizes.
trainsize = int(df2.shape[0] / 2)
df_test = df2.iloc[trainsize:, :].copy()
df_train = df2.iloc[:trainsize, :].copy()

Y_train = df_train['VALUE>400K'].values.reshape(-1, 1)
X_train = df_train.loc[:, ~(df_train.columns).isin(['VALUE>400K'])]
Y_test = df_test['VALUE>400K'].values.reshape(-1, 1)
X_test = df_test.loc[:, ~(df_test.columns).isin(['VALUE>400K'])]

regr1 = linear_model.LinearRegression()
Y_train = df_train['VALUE>400K'].values.reshape(-1, 1)
X_train = df_train.loc[:, ~(df_train.columns).isin(['VALUE>400K'])]
regr1.fit(X_train, Y_train)
print('\nR-squared:')
print(regr1.score(X_train, Y_train))

regrorigparams = np.append(regr1.coef_, regr1.intercept_)
print('\nParameter estimates:')
print(regrorigparams)

regr_scores = cross_val_score(regr1, X_train, Y_train, cv=5)
print('\nPercentage accuracy')
print(regr_scores)
print('Mean:', regr_scores.mean())



R-squared:
0.5661126724802301

Parameter estimates:
[-7.47650145e-08 -1.04212280e-05  3.24308294e-05 -4.38763001e-05
 -4.25386754e-05  1.45261408e-04  3.97816794e-05  9.00034546e-05
  9.90937787e-02  1.17058905e-04  4.04171328e-04  3.34126726e-05
 -2.03976270e-02  2.30366455e-06 -1.37402695e-05 -3.87097063e-04
  3.97699324e-04  3.87091732e-04 -9.77599192e-05  5.05558889e-02
 -4.52839155e-13 -3.99885343e-02  2.62047328e-14 -2.70181104e-02
 -1.38582109e-01 -9.15933995e-16  0.00000000e+00  0.00000000e+00
  1.38341873e-01  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -5.36759852e-02 -7.65798544e-03  0.00000000e+00 -1.57786552e-01
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  2.86367403e-01  0.00000000e+00 -9.12780992e-01]

Percentage accuracy
[0.06621722 0.61649004 0.48515387 0.36298379 0.26486312]
Mean: 0.35914160640536635


In [143]:
#Ridge Regression
ridgeregr = linear_model.Ridge(alpha=10, fit_intercept=False) 
ridgeregr.fit(X_train, Y_train)
rrorigparams = ridgeregr.coef_[0]

print('\nR-squared:')
print(ridgeregr.score(X_train, Y_train))
print('\nParameter Estimates:')
print(rrorigparams)

regr_scores = cross_val_score(ridgeregr, X_train, Y_train, cv=5)
print('\nPercentage accuracy')
print(regr_scores)
print('Mean:', regr_scores.mean())


R-squared:
0.5657850687420449

Parameter Estimates:
[-2.19306698e-07 -1.02054361e-05  3.08382957e-05 -4.35322914e-05
 -4.20800937e-05  1.45741106e-04  3.83580259e-05  9.04210544e-05
  9.91276097e-02 -9.12663561e-04  8.40624813e-05  3.07942677e-05
 -2.72334424e-02  2.26034408e-06 -1.36655019e-05 -3.97968219e-04
  4.08588944e-04  3.98114334e-04 -1.15985219e-04  5.60922410e-02
  0.00000000e+00 -4.96052870e-02  0.00000000e+00 -2.86700394e-01
 -9.29127442e-02  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -9.78514380e-03  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -3.17343704e-01 -2.77004019e-01  0.00000000e+00 -3.79430891e-02
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  9.40944251e-04  0.00000000e+00]

Percentage accuracy
[0.08522649 0.61542365 0.48333381 0.36183686 0.29481246]
Mean: 0.3681266547227506


# Lasso Regression

In [136]:

lass = linear_model.Lasso(alpha=.35)
lassfit = lass.fit(X_train, Y_train)
print('R²:')
print(lass.score(X_train, Y_train))
origparams = np.append(lassfit.coef_, lassfit.intercept_)
print('\nParameter estimates:')
print(origparams)

lass_scores = cross_val_score(lass, X_train, Y_train, cv=5)
print('\nPercentage accuracy')
print(lass_scores)
print('Mean:', lass_scores.mean())


R²:
0.550669112712775

Parameter estimates:
[ 4.19137866e-08 -8.90529218e-06  3.10243208e-05 -4.81233859e-05
  7.92421662e-05  1.27241736e-04  7.71792720e-05  8.28674464e-05
  0.00000000e+00  0.00000000e+00  5.22882669e-04  2.80860073e-05
  0.00000000e+00  2.17288124e-06 -1.17370750e-05  8.65433799e-06
  1.47174508e-06 -1.01934804e-05 -1.36855176e-04  0.00000000e+00
  0.00000000e+00 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
 -0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00 -0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.07012972e+00]

Percentage accuracy
[0.16301462 0.61054124 0.45587102 0.33405973 0.28153156]
Mean: 0.36900363145077736


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [132]:
print(lass.score(X_test, Y_test))

0.13698546734997996


The model is not great, with a lot of variability in percent accuracy. The a